In [ ]:
import pandas as pd

## 1. Entendimento e Limpeza da Base

In [ ]:
df = pd.read_csv('ecommerce_orders.csv')

In [ ]:
df.head(20)


In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.dtypes

In [ ]:
df['price'] = df['price'].astype(float)
df['quantity'] = df['quantity'].astype(float)

In [ ]:
valores_menor_que_zero = (df['price']< 0).sum()
valores_menor_que_zero

In [ ]:
valores_zerados = (df['price']== 0).sum()
valores_zerados

In [ ]:
df['order_date'] =  pd.to_datetime(df['order_date'])
df.info()

A base de e‑commerce contém 20 registros e 8 colunas. As colunas order_id, customer_id, product, category e payment_method são variáveis categóricas armazenadas como texto (object). As colunas price e quantity são numéricas (int64), representando respectivamente o preço unitário e a quantidade de itens por pedido. A coluna order_date está no formato de data (datetime64[ns]). Nenhuma coluna apresenta valores ausentes: todas possuem 20 registros não nulos. Dessa forma, a estrutura da base é simples e a qualidade dos dados é boa, sem faltas ou problemas de tipo aparentes nesta etapa inicial.

## 2. Criar Coluna de Valor Total do Pedido

In [ ]:
# Criando Coluna total pedido
df['total_amount'] = df['price'] * df['quantity'].astype(float)


In [ ]:
df.describe().round(2)

## 3. Visão Geral das Vendas

In [ ]:
valor_total_vendido = df['total_amount'].sum().round(2)
valor_total_vendido

In [ ]:
numero_pedido = df['order_id'].count().round(2)
numero_pedido

In [ ]:
ticket_medio = valor_total_vendido/numero_pedido
ticket_medio

In [ ]:
media_itens_por_pedido = df['quantity'].mean()
media_itens_por_pedido

In [ ]:
resumo_metricas = pd.DataFrame({
    'métrica': [
        'Número de pedidos',
        'Média de itens por pedido',
        'Ticket médio',
        'Valor total vendido'
    ],
    'valor': [
        numero_pedido,
        media_itens_por_pedido,
        ticket_medio,
        valor_total_vendido
    ]
})

resumo_metricas


## 4. Análise por Categoria de Produto

In [ ]:
numero_vendas_categoria = df.groupby('category')[['quantity', 'payment_method']].count()
numero_vendas_categoria

In [ ]:
faturamento_categoria = df.groupby('category')['total_amount'].sum().sort_values(ascending=False)
faturamento_categoria

In [ ]:
quantidade_vendidos_categoria = df.groupby('category')['quantity'].sum()
quantidade_vendidos_categoria

In [ ]:
media_categoria = df.groupby('category')['total_amount'].mean().round(2)
media_categoria

In [ ]:
faturamento_product = df.groupby('product')['total_amount'].sum().sort_values(ascending=False)
faturamento_product

In [ ]:
# Categoria com maior quantidade de itens vendidos
cat_mais_itens = quantidade_vendidos_categoria.idxmax()
qtd_mais_itens = quantidade_vendidos_categoria.max()

# Categoria com maior faturamento
cat_mais_faturamento = faturamento_categoria.idxmax()
valor_mais_faturamento = faturamento_categoria.max()

# Produto com maior faturamento
prod_mais_faturamento = faturamento_product.idxmax()
valor_prod_mais_faturamento = faturamento_product.max()

resumo_faturamento = pd.DataFrame({
    'faturamento': [
        'Categoria com maior quantidade',
        'Categoria mais vendida (R$)',
        'Produto mais vendido (R$)',
    ],
    'nome': [
        cat_mais_itens,
        cat_mais_faturamento,
        prod_mais_faturamento,
    ],
    'valor': [
        qtd_mais_itens,
        valor_mais_faturamento,
        valor_prod_mais_faturamento,
    ]
})

resumo_faturamento


## 5. Análise de Produtos Individuais

In [ ]:
# FIltrando todos os dados por produto, produto selecionado fica dentro de  .get_group('Shoes')
dados_produto = df.groupby('product').get_group('Shoes')
dados_produto

In [ ]:
faturamento_product = df.groupby('product')['total_amount'].mean().sort_values(ascending=False).round(2)
faturamento_product

## 6. Análise Temporal (Mês e Dia da Semana)

In [ ]:
df['data'] = df['order_date'].dt.day_of_year
df['dia_semana'] = df['order_date'].dt.day_name()
df['mês'] = df['order_date'].dt.month
df['ano'] = df['order_date'].dt.year
df

In [ ]:
dia_vendas= df.groupby('data')['total_amount'].sum().sort_values(ascending=False)
dia_vendas

In [ ]:
dia_vendas= df.groupby('dia_semana')['total_amount'].sum().sort_values(ascending=False)
dia_vendas

In [ ]:
df.groupby('category').agg({
    'total_amount': ['max', 'min', 'mean'],
    'quantity': 'sum'
}).round(2)


## 7. Análise de Formas de Pagamento

In [ ]:
metodo_pagamento = df.groupby('payment_method') ['order_id'].count().sort_values(ascending=False)
metodo_pagamento

In [ ]:
soma_pagamento = df.groupby('payment_method') ['total_amount'].sum().sort_values(ascending=False)
soma_pagamento

In [ ]:
# metodo_pagamento  -> Series com a quantidade por método
# soma_pagamento    -> Series com a receita por método

participacao = pd.DataFrame({
    'qtd_pedidos': metodo_pagamento,
    'receita': soma_pagamento
})

participacao['perc_qtd'] = participacao['qtd_pedidos'] / participacao['qtd_pedidos'].sum() * 100
participacao['perc_receita'] = participacao['receita'] / participacao['receita'].sum() * 100

participacao


In [ ]:
count_pagamento = df.groupby('payment_method')['total_amount'].count()
count_pagamento

In [ ]:
soma_pagamento = df.groupby('payment_method')['total_amount'].sum()

count_pagamento = df.groupby('payment_method')['total_amount'].count()


ticket_medio_pagamento = (soma_pagamento/count_pagamento).sort_values(ascending=False)
ticket_medio_pagamento

In [ ]:
ticket_medio_pagamento = df.groupby('payment_method')['total_amount'].mean().round(2).sort_values(ascending=False)
ticket_medio_pagamento

In [ ]:
analise_formas_pagamento = pd.DataFrame ({
    'qtde_metodo_usado': count_pagamento,
    'receita_metodo_usado': soma_pagamento,
    'ticket_medio_metodo':ticket_medio_pagamento
    }).round(2).sort_values(by='receita_metodo_usado', ascending=False)
analise_formas_pagamento

In [ ]:
analise_formas_pagamento = pd.DataFrame({
    'qtde_metodo_usado': count_pagamento,
    'receita_metodo_usado': soma_pagamento,
    'ticket_medio_metodo': ticket_medio_pagamento
}).sort_index()


## 8. Insights e Recomendações de Negócio




Exemplos de conclusões:
Qual categoria/produto é a estrela
Qual mês é o melhor para vendas (sazonalidade)
Qual forma de pagamento é a mais comum
Qual é o perfil do cliente mais valioso
Oportunidades de melhoria (categorias/produtos com baixo desempenho)
# Propor 3-5 recomendações de ações que o negócio poderia tomar